- Sentiment 모듈 사용법

## 문장 추출

In [ ]:
import pandas as pd
import Sentiment
import sqlite3
# new_words_var.xlsx에서 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# sentence 추출
analysis = Sentiment.Sentiment()
sent = analysis.extract_sent(head, new_words)
# DataFrame(head)과 신어(words) 입력하면 신어에 해당하는 sentences 추출
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)
# 신어와 신어가 들어있는 예문을 sent.db로 저장

## 감성 사전 업데이트 및 감성 점수 추출

In [ ]:
# Initialize
analysis = Sentiment.Sentiment()
conn = sqlite3.connect('sent.db')
sent = pd.read_sql('SELECT * FROM sent', conn)
mapping_most_related = analysis.cal_score(sent)
# 신어-문장으로 된 DataFrame(sent) 입력하면 sentiment 점수를 excel에 저장하고 신조어와 most_related (단어, pmi) 쌍을 출력

positive = []
negative = []
for _ in tqdm(range(10)):
    # 일정 (단어-감성score) set을 사용하여, 감성사전 score 업데이트 (학습과정)
    sentiment_result = pd.read_excel('sentiment_result.xlsx') # sentiment_result.xlsx : 감성 점수 엑셀 파일
    positive, negative = analysis.update_score(positive, negative, sentiment_result) 
    # 감성사전 score 업데이트하고 업데이트된 감성 점수를 엑셀로 저장(sentiment_result.xlsx가 업데이트 됨)
    
    # 감성사전을 이용하여 감성분석을 진행
    sentiment = pd.read_excel('sentiment.xlsx')
    analysis.sentiment_analysis(sentiment, mapping_most_related)
    # 업데이트된 감성 점수를 이용하여 감성분석을 다시 함